In [73]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

### 获取初始页面的内容

In [2]:
html = urlopen("http://detail.zol.com.cn/cell_phone_index/subcate57_613_list_1.html")
if html.headers.get_content_charset() is None:
    bs = BeautifulSoup(html.read(), 'html.parser')
else:
    bs = BeautifulSoup(html.read().decode(html.headers.get_content_charset()), 'html.parser')

### 获取内容里的所有手机种类连接

In [14]:
list_phone_name = []
list_phone_link = []

list_phone = all_phone_link = bs.find_all('a', {'href':re.compile('^/cell_phone_index/subcate57_[0-9]+_list_1\.html'), 
                                   'data-link':re.compile('[0-9]+')})
for current_phone in list_phone:
    list_phone_name.append(current_phone.get_text())
    list_phone_link.append('http://detail.zol.com.cn' + '' +current_phone['href'])
list_phone

[<a class="active" data-letter="H" data-link="613" href="/cell_phone_index/subcate57_613_list_1.html">华为</a>,
 <a data-letter="O" data-link="1673" href="/cell_phone_index/subcate57_1673_list_1.html">OPPO</a>,
 <a data-letter="V" data-link="1795" href="/cell_phone_index/subcate57_1795_list_1.html">vivo</a>,
 <a data-letter="P" data-link="544" href="/cell_phone_index/subcate57_544_list_1.html">苹果</a>,
 <a data-letter="S" data-link="98" href="/cell_phone_index/subcate57_98_list_1.html">三星</a>,
 <a data-letter="R" data-link="50840" href="/cell_phone_index/subcate57_50840_list_1.html">荣耀</a>,
 <a data-letter="X" data-link="34645" href="/cell_phone_index/subcate57_34645_list_1.html">小米</a>,
 <a data-letter="Y" data-link="35579" href="/cell_phone_index/subcate57_35579_list_1.html">一加</a>,
 <a data-letter="M" data-link="1434" href="/cell_phone_index/subcate57_1434_list_1.html">魅族</a>,
 <a data-letter="R" data-link="55535" href="/cell_phone_index/subcate57_55535_list_1.html">realme</a>,
 <a dat

### 找到所有种类所有手机列表的连接

In [39]:
list_current_phone_link = []
for current_phone_link in list_phone_link:
    html = urlopen(current_phone_link)
    list_current_phone_link.append(current_phone_link)
    if html.headers.get_content_charset() is None:
        sub_bs = BeautifulSoup(html.read(), 'html.parser')
    else:
        sub_bs = BeautifulSoup(html.read().decode(html.headers.get_content_charset()), 'html.parser')
    current_link_prefix = current_phone_link.replace(".html", "")
    current_link_prefix = current_link_prefix.replace("http://detail.zol.com.cn", "")
    current_phone_all_link = sub_bs.find_all('a', {'href':re.compile('^'+current_link_prefix+'_.*'),}, text=re.compile('^[0-9]$'))
    for i in current_phone_all_link:
        list_current_phone_link.append('http://detail.zol.com.cn' + '' +i['href'])

### 获取所有手机的链接地址

In [56]:
list_link = []
for current_phone_link in list_current_phone_link:
    html = urlopen(current_phone_link)
    if html.headers.get_content_charset() is None:
        sub_bs = BeautifulSoup(html.read(), 'html.parser')
    else:
        sub_bs = BeautifulSoup(html.read().decode(html.headers.get_content_charset()), 'html.parser')
    for i in sub_bs.find_all('a', {'class': 'pic'}, target='_blank'):
        if len(i.attrs) == 3:
            list_link.append('http://detail.zol.com.cn' + '' +i['href'])

### 获取所有数据

In [113]:
column_list = ['生产商', '手机名称', '手机价格', '手机排名', '手机别名']
output_file = "C:\\Users\\hao.ren3\\Desktop\\phone_price.csv"

exception_file = "C:\\Users\\hao.ren3\\Desktop\\exception_price.csv"
exception_data = pd.DataFrame(columns=['url'])
exception_data.to_csv(exception_file, index=None)

data = pd.DataFrame(columns=column_list)
data.to_csv(output_file, index=None, encoding="utf_8_sig", quoting=1)

visited_file = "C:\\Users\\hao.ren3\\Desktop\\visited_page.csv"
visited_page = pd.read_csv(visited_file, header=0)

for phone_link in list_link:
    html = urlopen(phone_link)
    if phone_link in visited_page['page'].values:
        continue
    
    print(phone_link)
    try:
        if html.headers.get_content_charset() is None:
            sub_bs = BeautifulSoup(html.read(), 'html.parser')
        else:
            sub_bs = BeautifulSoup(html.read().decode(html.headers.get_content_charset()), 'html.parser')

        marks = sub_bs.find_all('a', {'class':'breadcrumb__manu'})
        mark = ''
        if len(marks) > 0:
            mark = marks[0].get_text()

        phone_names = sub_bs.find_all('h1', {'class':'product-model__name'})
        phone_name = ''
        if len(phone_names) > 0:
            phone_name = phone_names[0].get_text()

        phone_ranks = sub_bs.find_all('a', {'class':'product-model__rank'})
        phone_rank = ''
        if len(phone_ranks) > 0:
            phone_rank = phone_ranks[0].get_text()

        other_names = sub_bs.find_all('h2', {'class':'product-model__alias'})
        other_name = ''
        if len(other_names) > 0:
            other_name = other_names[0].get_text().replace("别名：", "")

        phone_prices = sub_bs.find_all('b', {'class':'price-type'})
        phone_price = ''
        if len(phone_prices) > 0:
            phone_price = phone_prices[0].get_text()

        current_data = pd.DataFrame(np.reshape([str(mark), str(phone_name), str(phone_price), str(phone_rank), str(other_name)], (1, -1)))
        current_data.to_csv(output_file, index=None, header=None, mode='a', quoting=1, encoding="utf_8_sig")

        page_web = pd.DataFrame([phone_link])
        page_web.to_csv(visited_file, mode='a', index=None, header=None)
    except UnicodeDecodeError as e:
        current_exception = pd.DataFrame([phone_link])
        current_exception.to_csv(exception_file, index=None, header=None, mode='a')
        page_web = pd.DataFrame([phone_link])
        page_web.to_csv(visited_file, mode='a', index=None, header=None)

http://detail.zol.com.cn/cell_phone/index42550.shtml
http://detail.zol.com.cn/cell_phone/index314064.shtml
http://detail.zol.com.cn/cell_phone/index373979.shtml
http://detail.zol.com.cn/cell_phone/index70865.shtml
http://detail.zol.com.cn/cell_phone/index70867.shtml
http://detail.zol.com.cn/cell_phone/index70868.shtml
http://detail.zol.com.cn/cell_phone/index399268.shtml
http://detail.zol.com.cn/cell_phone/index370999.shtml
http://detail.zol.com.cn/cell_phone/index373372.shtml
http://detail.zol.com.cn/cell_phone/index373498.shtml
http://detail.zol.com.cn/cell_phone/index343211.shtml
http://detail.zol.com.cn/cell_phone/index343217.shtml
http://detail.zol.com.cn/cell_phone/index343226.shtml
http://detail.zol.com.cn/cell_phone/index343228.shtml
http://detail.zol.com.cn/cell_phone/index44611.shtml
http://detail.zol.com.cn/cell_phone/index17460.shtml
http://detail.zol.com.cn/cell_phone/index17465.shtml
http://detail.zol.com.cn/cell_phone/index17473.shtml
http://detail.zol.com.cn/cell_phone/

http://detail.zol.com.cn/cell_phone/index31980.shtml
http://detail.zol.com.cn/cell_phone/index31981.shtml
http://detail.zol.com.cn/cell_phone/index31982.shtml
http://detail.zol.com.cn/cell_phone/index32001.shtml
http://detail.zol.com.cn/cell_phone/index1270696.shtml
http://detail.zol.com.cn/cell_phone/index1270784.shtml
http://detail.zol.com.cn/cell_phone/index1202452.shtml
http://detail.zol.com.cn/cell_phone/index1270735.shtml
http://detail.zol.com.cn/cell_phone/index1271100.shtml
http://detail.zol.com.cn/cell_phone/index1175232.shtml
http://detail.zol.com.cn/cell_phone/index1270801.shtml
http://detail.zol.com.cn/cell_phone/index1185264.shtml
http://detail.zol.com.cn/cell_phone/index1207384.shtml
http://detail.zol.com.cn/cell_phone/index1185338.shtml
http://detail.zol.com.cn/cell_phone/index1270996.shtml
http://detail.zol.com.cn/cell_phone/index1271058.shtml
http://detail.zol.com.cn/cell_phone/index1174799.shtml
http://detail.zol.com.cn/cell_phone/index1141873.shtml
http://detail.zol.

http://detail.zol.com.cn/cell_phone/index248810.shtml
http://detail.zol.com.cn/cell_phone/index186717.shtml
http://detail.zol.com.cn/cell_phone/index218511.shtml
http://detail.zol.com.cn/cell_phone/index1234072.shtml
http://detail.zol.com.cn/cell_phone/index391796.shtml
http://detail.zol.com.cn/cell_phone/index1165361.shtml
http://detail.zol.com.cn/cell_phone/index1169493.shtml
http://detail.zol.com.cn/cell_phone/index1209063.shtml
http://detail.zol.com.cn/cell_phone/index1165359.shtml
http://detail.zol.com.cn/cell_phone/index1151235.shtml
http://detail.zol.com.cn/cell_phone/index375871.shtml
http://detail.zol.com.cn/cell_phone/index1163093.shtml
http://detail.zol.com.cn/cell_phone/index384124.shtml
http://detail.zol.com.cn/cell_phone/index374263.shtml
http://detail.zol.com.cn/cell_phone/index391402.shtml
http://detail.zol.com.cn/cell_phone/index350261.shtml
http://detail.zol.com.cn/cell_phone/index1178659.shtml
http://detail.zol.com.cn/cell_phone/index1101949.shtml
http://detail.zol.c

http://detail.zol.com.cn/cell_phone/index291161.shtml
http://detail.zol.com.cn/cell_phone/index1273644.shtml
http://detail.zol.com.cn/cell_phone/index1273659.shtml
http://detail.zol.com.cn/cell_phone/index1273657.shtml
http://detail.zol.com.cn/cell_phone/index1273616.shtml
http://detail.zol.com.cn/cell_phone/index1273601.shtml
http://detail.zol.com.cn/cell_phone/index1273654.shtml
http://detail.zol.com.cn/cell_phone/index1205834.shtml
http://detail.zol.com.cn/cell_phone/index1251517.shtml
http://detail.zol.com.cn/cell_phone/index1251501.shtml
http://detail.zol.com.cn/cell_phone/index1251524.shtml
http://detail.zol.com.cn/cell_phone/index1260417.shtml
http://detail.zol.com.cn/cell_phone/index1260420.shtml
http://detail.zol.com.cn/cell_phone/index1260386.shtml
http://detail.zol.com.cn/cell_phone/index1260363.shtml
http://detail.zol.com.cn/cell_phone/index1260376.shtml
http://detail.zol.com.cn/cell_phone/index1260430.shtml
http://detail.zol.com.cn/cell_phone/index1251512.shtml
http://deta

http://detail.zol.com.cn/cell_phone/index1185360.shtml
http://detail.zol.com.cn/cell_phone/index1176427.shtml
http://detail.zol.com.cn/cell_phone/index1208156.shtml
http://detail.zol.com.cn/cell_phone/index1185345.shtml
http://detail.zol.com.cn/cell_phone/index1259897.shtml
http://detail.zol.com.cn/cell_phone/index1259881.shtml
http://detail.zol.com.cn/cell_phone/index1259864.shtml
http://detail.zol.com.cn/cell_phone/index1179570.shtml
http://detail.zol.com.cn/cell_phone/index1204304.shtml
http://detail.zol.com.cn/cell_phone/index1207787.shtml
http://detail.zol.com.cn/cell_phone/index1139112.shtml
http://detail.zol.com.cn/cell_phone/index1167640.shtml
http://detail.zol.com.cn/cell_phone/index1163244.shtml
http://detail.zol.com.cn/cell_phone/index1216732.shtml
http://detail.zol.com.cn/cell_phone/index1215086.shtml
http://detail.zol.com.cn/cell_phone/index1150895.shtml
http://detail.zol.com.cn/cell_phone/index1162342.shtml
http://detail.zol.com.cn/cell_phone/index1234512.shtml
http://det

http://detail.zol.com.cn/cell_phone/index1226232.shtml
http://detail.zol.com.cn/cell_phone/index1213137.shtml
http://detail.zol.com.cn/cell_phone/index1205822.shtml
http://detail.zol.com.cn/cell_phone/index1208902.shtml
http://detail.zol.com.cn/cell_phone/index1226255.shtml
http://detail.zol.com.cn/cell_phone/index1180919.shtml
http://detail.zol.com.cn/cell_phone/index1226225.shtml
http://detail.zol.com.cn/cell_phone/index1226252.shtml
http://detail.zol.com.cn/cell_phone/index395901.shtml
http://detail.zol.com.cn/cell_phone/index277957.shtml
http://detail.zol.com.cn/cell_phone/index364877.shtml
http://detail.zol.com.cn/cell_phone/index367166.shtml
http://detail.zol.com.cn/cell_phone/index1175452.shtml
http://detail.zol.com.cn/cell_phone/index310494.shtml
http://detail.zol.com.cn/cell_phone/index280457.shtml
http://detail.zol.com.cn/cell_phone/index392509.shtml
http://detail.zol.com.cn/cell_phone/index392511.shtml
http://detail.zol.com.cn/cell_phone/index304793.shtml
http://detail.zol.c

http://detail.zol.com.cn/cell_phone/index1261689.shtml
http://detail.zol.com.cn/cell_phone/index1136714.shtml
http://detail.zol.com.cn/cell_phone/index395161.shtml
http://detail.zol.com.cn/cell_phone/index1211769.shtml
http://detail.zol.com.cn/cell_phone/index1136874.shtml
http://detail.zol.com.cn/cell_phone/index384896.shtml
http://detail.zol.com.cn/cell_phone/index1202688.shtml
http://detail.zol.com.cn/cell_phone/index1260970.shtml
http://detail.zol.com.cn/cell_phone/index394837.shtml
http://detail.zol.com.cn/cell_phone/index394838.shtml
http://detail.zol.com.cn/cell_phone/index366252.shtml
http://detail.zol.com.cn/cell_phone/index393632.shtml
http://detail.zol.com.cn/cell_phone/index369290.shtml
http://detail.zol.com.cn/cell_phone/index392838.shtml
http://detail.zol.com.cn/cell_phone/index398164.shtml
http://detail.zol.com.cn/cell_phone/index398329.shtml
http://detail.zol.com.cn/cell_phone/index373870.shtml
http://detail.zol.com.cn/cell_phone/index374056.shtml
http://detail.zol.com.

http://detail.zol.com.cn/cell_phone/index199767.shtml
http://detail.zol.com.cn/cell_phone/index259792.shtml
http://detail.zol.com.cn/cell_phone/index325518.shtml
http://detail.zol.com.cn/cell_phone/index295523.shtml
http://detail.zol.com.cn/cell_phone/index71987.shtml
http://detail.zol.com.cn/cell_phone/index163193.shtml
http://detail.zol.com.cn/cell_phone/index47646.shtml
http://detail.zol.com.cn/cell_phone/index319286.shtml
http://detail.zol.com.cn/cell_phone/index137096.shtml
http://detail.zol.com.cn/cell_phone/index137098.shtml
http://detail.zol.com.cn/cell_phone/index137336.shtml
http://detail.zol.com.cn/cell_phone/index105209.shtml
http://detail.zol.com.cn/cell_phone/index1156284.shtml
http://detail.zol.com.cn/cell_phone/index235614.shtml
http://detail.zol.com.cn/cell_phone/index328216.shtml
http://detail.zol.com.cn/cell_phone/index27741.shtml
http://detail.zol.com.cn/cell_phone/index27743.shtml
http://detail.zol.com.cn/cell_phone/index27745.shtml
http://detail.zol.com.cn/cell_ph

http://detail.zol.com.cn/cell_phone/index1150094.shtml
http://detail.zol.com.cn/cell_phone/index1150060.shtml
http://detail.zol.com.cn/cell_phone/index1148800.shtml
http://detail.zol.com.cn/cell_phone/index1182019.shtml
http://detail.zol.com.cn/cell_phone/index396124.shtml
http://detail.zol.com.cn/cell_phone/index396125.shtml
http://detail.zol.com.cn/cell_phone/index394597.shtml
http://detail.zol.com.cn/cell_phone/index369460.shtml
http://detail.zol.com.cn/cell_phone/index369462.shtml
http://detail.zol.com.cn/cell_phone/index367023.shtml
http://detail.zol.com.cn/cell_phone/index1205572.shtml
http://detail.zol.com.cn/cell_phone/index392575.shtml
http://detail.zol.com.cn/cell_phone/index368786.shtml
http://detail.zol.com.cn/cell_phone/index368790.shtml
http://detail.zol.com.cn/cell_phone/index339709.shtml
http://detail.zol.com.cn/cell_phone/index339860.shtml
http://detail.zol.com.cn/cell_phone/index401163.shtml
http://detail.zol.com.cn/cell_phone/index372487.shtml
http://detail.zol.com.c